In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Set up Firefox options
firefox_options = Options()
firefox_options.add_argument("--headless")  # Run in headless mode (no browser UI)
firefox_options.add_argument("--no-sandbox")
firefox_options.add_argument("--disable-dev-shm-usage")

# Initialize the Firefox driver
driver = webdriver.Firefox(options=firefox_options)

# URL to scrape
url = "https://www.airbnb.com/s/Belo-Horizonte--MG/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&channel=EXPLORE&place_id=ChIJMyzPysqQpgARlznSOl55NVs&acp_id=8949c462-3712-48e3-8bed-185932a32fe1&date_picker_type=calendar&source=structured_search_input_header&search_type=user_map_move&query=Belo%20Horizonte%2C%20MG&search_mode=regular_search&price_filter_num_nights=1&ne_lat=-19.762079585687626&ne_lng=-43.924126024396315&sw_lat=-19.837703838936648&sw_lng=-43.99818231236125&zoom=12.883832463880852&zoom_level=12.883832463880852&search_by_map=true&checkin=2025-03-01&checkout=2025-03-02&monthly_start_date=2025-04-01&monthly_length=3&monthly_end_date=2025-07-01"

# Navigate to the URL
driver.get(url)

# Wait for the page to load
time.sleep(5)

# Wait for listings to appear
wait = WebDriverWait(driver, 10)
listings = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[itemprop='itemListElement']")))

# Initialize lists to store data
names = []
prices = []
ratings = []
details = []

# Extract information from each listing
for listing in listings:
    try:
        # Get listing name
        name = listing.find_element(By.CSS_SELECTOR, "meta[itemprop='name']").get_attribute("content")
        names.append(name)
    except:
        names.append("N/A")
    
    try:
        # Get price
        price = listing.find_element(By.CSS_SELECTOR, "span._hb913q").text
        prices.append(price)
    except:
        prices.append("N/A")
    
    try:
        # Get rating
        rating = listing.find_element(By.CSS_SELECTOR, "span[aria-label*='rated']").get_attribute("aria-label")
        ratings.append(rating)
    except:
        ratings.append("N/A")
    
    try:
        # Get details (beds, bedrooms, etc.)
        detail = listing.find_element(By.CSS_SELECTOR, "div[data-testid='listing-card-subtitle']").text
        details.append(detail)
    except:
        details.append("N/A")

# Create a DataFrame
airbnb_data = pd.DataFrame({
    'Name': names,
    'Price': prices,
    'Rating': ratings,
    'Details': details
})

# Display the data
print(f"Found {len(airbnb_data)} listings")
airbnb_data.head()

# Close the browser
driver.quit()



TimeoutException: Message: 


In [4]:
print(airbnb_data)

                                                 Name   Price Rating  \
0    Suite w/5-seat hydromassage,TV 75”w/sound cinema  R$342     N/A   
1              housespacebh(HEATED 35 DEGREES COVERED  R$643     N/A   
2                          Apto Floramar/Green plants  R$122     N/A   
3    Studio Enchantador in Belo Horizonte com Garagem  R$114     N/A   
4         Duplex MIAMI (Coverage) Administrative City   R$71     N/A   
5   Apto.Ótima Localiz. Prox.CenterMinas CidAdmin,...  R$132     N/A   
6     Aconchegante ap 500 meters da Lagoa da Pampulha  R$132     N/A   
7                                       Aurora Refuge  R$210     N/A   
8   GORGEOUS AP 800 METERS FROM THE LAGOON, NEAR UFMG  R$137     N/A   
9   Best Reception and Fine Tract is in the Nardél...   R$97     N/A   
10                                  Casa Santa Monica  R$171     N/A   
11      House Pampulha Sell New Air Conditioning Wifi  R$103     N/A   
12                      Apt Nex Pampulha Sell New Wif   R$92    

In [8]:
url = "https://www.airbnb.com/rooms/48697908?check_in=2025-04-02&check_out=2025-04-03"  # Reduced URL with only check-in and check-out dates


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Set up Firefox options
firefox_options = Options()
firefox_options.add_argument("--headless")  # Run in headless mode (no browser UI)
firefox_options.add_argument("--no-sandbox")
firefox_options.add_argument("--disable-dev-shm-usage")
firefox_options.add_argument("--window-size=1920,1080")  # Set window size

# Initialize the Firefox driver
driver = webdriver.Firefox(options=firefox_options)


try:
    # Navigate to the URL
    driver.get(url)

    # Wait longer for the page to load
    print("Waiting for page to load...")
    time.sleep(10)

    # Create a dictionary to store listing data
    listing_data = {}
    
    # Get listing title
    try:
        title_element = driver.find_element(By.CSS_SELECTOR, "h1")
        listing_data['title'] = title_element.text
    except Exception as e:
        listing_data['title'] = "N/A"
    
    # Get price
    try:
        # Try multiple possible price selectors
        price_selectors = [
            "span._hb913q", 
            "span._tyxjp1",  # Adding this selector based on the image
            "span[data-testid='price-element']",
            "div._1k4xcdh span",  # Common price container
            "span._1k4xcdh",      # Direct span with class _1k4xcdh
            "div._1qs94rc span"   # Another common price container
        ]
        
        price_found = False
        for selector in price_selectors:
            try:
                price_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in price_elements:
                    if element.text and "$" in element.text:
                        listing_data['price'] = element.text
                        price_found = True
                        break
                if price_found:
                    break
            except:
                continue
                
        if not price_found:
            # Last resort: look for any element containing a dollar sign
            elements = driver.find_elements(By.XPATH, "//*[contains(text(), '$')]")
            if elements:
                listing_data['price'] = elements[0].text
            else:
                listing_data['price'] = "N/A"
    except Exception as e:
        print(f"Error getting price: {e}")
        listing_data['price'] = "N/A"
    
    # Get cleaning fee
    try:
        # Try multiple approaches to find the cleaning fee
        # First, look for elements containing "cleaning fee" text
        cleaning_fee_elements = driver.find_elements(By.XPATH, 
            "//*[contains(text(), 'cleaning fee') or contains(text(), 'Cleaning fee')]")
        
        if cleaning_fee_elements:
            for element in cleaning_fee_elements:
                # Try to find the price near the cleaning fee text
                try:
                    # Look at parent/sibling elements for price
                    parent_div = element.find_element(By.XPATH, "./ancestor::div[contains(@class, '_1827zqi') or contains(@class, '_14omfj')]")
                    price_span = parent_div.find_element(By.CSS_SELECTOR, "span._1k4xcdh, span._1wxivx7, span._1ks8cgb")
                    listing_data['cleaning_fee'] = price_span.text
                    break
                except:
                    # Try looking at nearby elements
                    try:
                        # Look for price in the same row
                        row_div = element.find_element(By.XPATH, "./ancestor::div[position()=1 or position()=2 or position()=3]")
                        spans = row_div.find_elements(By.CSS_SELECTOR, "span")
                        for span in spans:
                            if "$" in span.text:
                                listing_data['cleaning_fee'] = span.text
                                break
                    except:
                        continue
        
        # If still not found, try a more general approach
        if 'cleaning_fee' not in listing_data or listing_data['cleaning_fee'] == "N/A":
            # Look for any div containing "cleaning fee" and find nearby price
            price_elements = driver.find_elements(By.XPATH, 
                "//div[contains(text(), 'cleaning fee') or contains(text(), 'Cleaning fee')]/following::span[contains(text(), '$')][1] | //div[contains(text(), 'cleaning fee') or contains(text(), 'Cleaning fee')]/parent::*/following-sibling::*//span[contains(text(), '$')]")
            if price_elements:
                listing_data['cleaning_fee'] = price_elements[0].text
            else:
                listing_data['cleaning_fee'] = "N/A"
    except Exception as e:
        print(f"Error getting cleaning fee: {e}")
        listing_data['cleaning_fee'] = "N/A"
    
    # Get bedrooms
    try:
        bedroom_element = driver.find_element(By.CSS_SELECTOR, "li.l7n4lsf:contains('bedroom')")
        listing_data['bedrooms'] = bedroom_element.text.strip()
    except Exception as e:
        try:
            # Alternative selector
            bedroom_elements = driver.find_elements(By.CSS_SELECTOR, "li.l7n4lsf")
            for element in bedroom_elements:
                if "bedroom" in element.text:
                    listing_data['bedrooms'] = element.text.strip()
                    break
            else:
                listing_data['bedrooms'] = "N/A"
        except:
            listing_data['bedrooms'] = "N/A"
    
    # Get beds
    try:
        bed_element = driver.find_element(By.CSS_SELECTOR, "li.l7n4lsf:contains('bed')")
        listing_data['beds'] = bed_element.text.strip()
    except Exception as e:
        try:
            # Alternative selector
            bed_elements = driver.find_elements(By.CSS_SELECTOR, "li.l7n4lsf")
            for element in bed_elements:
                if "bed" in element.text and "bedroom" not in element.text:
                    listing_data['beds'] = element.text.strip()
                    break
            else:
                listing_data['beds'] = "N/A"
        except:
            listing_data['beds'] = "N/A"
    
    # Get bathrooms
    try:
        bath_element = driver.find_element(By.CSS_SELECTOR, "li.l7n4lsf:contains('bath')")
        listing_data['bathrooms'] = bath_element.text.strip()
    except Exception as e:
        try:
            # Alternative selector
            bath_elements = driver.find_elements(By.CSS_SELECTOR, "li.l7n4lsf")
            for element in bath_elements:
                if "bath" in element.text:
                    listing_data['bathrooms'] = element.text.strip()
                    break
            else:
                listing_data['bathrooms'] = "N/A"
        except:
            listing_data['bathrooms'] = "N/A"
    
    # Create a DataFrame with the single listing
    listing_df = pd.DataFrame([listing_data])
    
    # Display the data
    print("Listing Information:")
    print(listing_df)
    
except Exception as e:
    print(f"An error occurred: {e}")
    # Take a screenshot to help debug
    try:
        driver.save_screenshot("airbnb_error.png")
        print("Screenshot saved as airbnb_error.png")
    except:
        print("Could not save screenshot")

finally:
    # Close the browser
    driver.quit()

Waiting for page to load...
Listing Information:
                                    title   price cleaning_fee     bedrooms  \
0  Flat decorated in the heart of Savassi  R$300                · 1 bedroom   

      beds bathrooms  
0  · 1 bed  · 1 bath  
